This notebook assesses the ability to predict demographic outcomes from survey data.  

In [150]:
import os,glob
import numpy,pandas
from sklearn.svm import LinearSVC,SVC
from sklearn.linear_model import LinearRegression,LogisticRegressionCV,RandomizedLogisticRegression,ElasticNet,ElasticNetCV,Ridge,RidgeCV
from sklearn.preprocessing import scale
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import StratifiedKFold,KFold
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score,classification_report,confusion_matrix
%matplotlib inline

%load_ext rpy2.ipython
%R require(mirt)

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


array([1], dtype=int32)

In [2]:
if not os.path.exists('factor_scores'):
    os.mkdir('factor_scores')


In [33]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [146]:
binary_vars=["ArrestedChargedLifeCount","DivorceCount","GamblingProblem","ChildrenNumber",
            "CreditCardDebt","RentOwn","RetirementAccount","TrafficTicketsLastYearCount","Obese",
             "TrafficAccidentsLifeCount","CaffienatedSodaCansPerDay"]
def get_demog_data(binarize=True):
    demogdata=pandas.read_csv('surveydata/demographics.tsv',index_col=0,delimiter='\t')
    # remove a couple of outliers
    demogdata=demogdata.query('WeightPounds>50')
    demogdata=demogdata.query('CaffienatedSodaCansPerDay>-1')
    demogdata=demogdata.assign(BMI=demogdata['WeightPounds']*0.45 / (demogdata['HeightInches']*0.025)**2)
    demogdata=demogdata.assign(Obese=(demogdata['BMI']>30).astype('int'))
    demogdata=demogdata[binary_vars]
    demogdata=demogdata.loc[demogdata.isnull().sum(1)==0]
    if binarize:
        for i in range(len(binary_vars)):
            v=binary_vars[i]
            if not demogdata[v].min()==0:
                demogdata.loc[demogdata[v]==demogdata[v].min(),v]=0
            demogdata.loc[demogdata[v]>demogdata[v].min(),v]=1
            assert demogdata[v].isnull().sum()==0



    return demogdata

def get_subscale_data():
    subscale_data=pandas.read_csv('survey_subscales.csv',index_col=0)
    subscale_data=subscale_data.ix[subscale_data.isnull().sum(1)==0]
    return subscale_data

def get_joint_dataset(d1,d2):
    d1_index=set(d1.index)
    d2_index=set(d2.index)
    inter=list(d1_index.intersection(d2_index))
    return d1.ix[inter],d2.ix[inter]
    return inter

demogdata,subscale_data=get_joint_dataset(get_demog_data(),get_subscale_data())
assert list(demogdata.index)==list(subscale_data.index)

#subscale_data=scale(subscale_data.values)

First get binary variables and test classification based on survey data.  Only include variables that have at least 10% of the infrequent category. Some of these were not collected as binary variables, but we binarize by calling anything above the minimum value a positive outcome.

In [92]:
nfeatures=5 # number of features to show
classifier='RandomForest'

def print_confusion_matrix(y_true,y_pred,labels=[0,1]):
    cm=confusion_matrix(y_true,y_pred)
    print('Confusion matrix')
    print('\t\tPredicted')
    print('\t\t0\t1')
    print('Actual\t0\t%d\t%d'%(cm[0,0],cm[0,1]))
    print('\t1\t%d\t%d'%(cm[1,0],cm[1,1]))


for i in range(len(binary_vars)):
    print('')
    y=bvardata[:,i]
    kf=StratifiedKFold(y,n_folds=8) # use stratified K-fold CV to get roughly equal folds
    # we use an inner CV loop on training data to estimate the best penalty value
    if classifier=='RandomForest':
        clf=RandomForestClassifier()
    elif classifier=='knn':
        clf=KNN()
    else:
        clf=SVC(probability=True) #LogisticRegressionCV(solver='liblinear',penalty='l1')  #LinearSVC()
    
    pred=numpy.zeros(len(y))
    pred_prob=numpy.zeros(len(y))


    for train,test in kf:
        clf.fit(subscale_data.iloc[train,:].values,y[train])
        if hasattr(clf,'predict_proba'):
            pred_prob.flat[test]=clf.predict_proba(subscale_data.iloc[test,:].values)
        pred.flat[test]=clf.predict(subscale_data.iloc[test,:].values)
    
    rocauc=roc_auc_score(y,pred_prob)
    print('%s:'%binary_vars[i])

    if numpy.var(pred)==0:
        print('WARNING: no variance in classifier output, degenerate model fit')
    if hasattr(clf,'predict_proba'):
        print('predictive accuracy (AUC: chance = 0.5) = %0.3f'%rocauc)
    else:
        print('Predictive accuracy')
    print(classification_report(y,pred,labels=[0,1]))
    print_confusion_matrix(y,pred)
    print("Features sorted by their absolute correlation with outcome (top %d):"%nfeatures)
    featcorr=numpy.array([numpy.corrcoef(subscale_data.iloc[:,x],y)[0,1] for x in range(subscale_data.shape[1])])
    idx=numpy.argsort(numpy.abs(featcorr))[::-1]
    for i in range(nfeatures):
        print('%f: %s'%(featcorr[idx[i]],subscale_vars[idx[i]]))


ArrestedChargedLifeCount:
predictive accuracy (AUC: chance = 0.5) = 0.605
             precision    recall  f1-score   support

          0       0.83      0.95      0.88       129
          1       0.00      0.00      0.00        26

avg / total       0.69      0.79      0.74       155

Confusion matrix
		Predicted
		0	1
Actual	0	123	6
	1	26	0
Features sorted by their absolute correlation with outcome (top 5):
-0.158825: future_time_perspective_survey.future_time_perspective
0.131977: dospert_eb_survey.recreational
0.129978: eating_survey.cognitive_restraint
0.128566: impulsive_venture_survey.venturesomeness
0.121823: bis_bas_survey.BAS_fun_seeking

DivorceCount:
predictive accuracy (AUC: chance = 0.5) = 0.481
             precision    recall  f1-score   support

          0       0.88      0.99      0.93       136
          1       0.00      0.00      0.00        19

avg / total       0.77      0.86      0.81       155

Confusion matrix
		Predicted
		0	1
Actual	0	134	2
	1	19	0
Featu

/Users/poldrack/anaconda/envs/py34/lib/python3.4/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


ChildrenNumber:
predictive accuracy (AUC: chance = 0.5) = 0.513
             precision    recall  f1-score   support

          0       0.61      0.74      0.67        96
          1       0.36      0.24      0.29        59

avg / total       0.52      0.55      0.52       155

Confusion matrix
		Predicted
		0	1
Actual	0	71	25
	1	45	14
Features sorted by their absolute correlation with outcome (top 5):
-0.306942: dospert_rp_survey.financial
-0.210904: bis_bas_survey.BAS_reward_responsiveness
0.201684: dospert_eb_survey.financial
0.179619: dospert_rt_survey.financial
-0.171689: dospert_rp_survey.health/safety

CreditCardDebt:
predictive accuracy (AUC: chance = 0.5) = 0.530
             precision    recall  f1-score   support

          0       0.55      0.72      0.62        89
          1       0.34      0.20      0.25        66

avg / total       0.46      0.50      0.46       155

Confusion matrix
		Predicted
		0	1
Actual	0	64	25
	1	53	13
Features sorted by their absolute correlation

In [ ]:
%%R -i workers
compnums=c(3:10)
for (i in 1:length(compnums)) {
  ncomps=compnums[i]
  load(sprintf('rdata_files_wrangler/mirt_%ddims.Rdata',ncomps))
  scores=fscores(m,full.scores = TRUE,method='MAP')
  scores=data.frame(scores)
  row.names(scores)=workers
  write.table(scores,file=sprintf('factor_scores/factor_scores_%ddims.tsv',ncomps),sep='\t',quote=FALSE,col.names=FALSE)
}

Now test using scores from MIRT - note that there is a bit of leakage here because the full dataset was used to estimate the MIRT models.  Ultimately we want to fit to discovery set and test on validation set.

In [148]:
ncomps=10
def get_mirt_data(ncomps=10):
    scoredata=pandas.read_csv('factor_scores/factor_scores_%ddims.tsv'%ncomps,delimiter='\t',index_col=0,header=None)
    scoredata=scoredata.loc[scoredata.isnull().sum(1)==0]
    return scoredata
demogdata,mirt_data=get_joint_dataset(get_demog_data(),get_mirt_data())
assert list(demogdata.index)==list(mirt_data.index)
mirt_vars=list(mirt_data.columns)
print(demogdata.shape)
print(mirt_data.shape)

(151, 11)
(151, 10)


In [149]:

for i in range(len(binary_vars)):
    print('')
    y=demogdata.loc[:,binary_vars[i]].values
    kf=StratifiedKFold(y,n_folds=8) # use stratified K-fold CV to get roughly equal folds
    # we use an inner CV loop on training data to estimate the best penalty value
    if classifier=='RandomForest':
        clf=RandomForestClassifier()
    elif classifier=='knn':
        clf=KNN()
    else:
        clf=SVC(probability=True) #LogisticRegressionCV(solver='liblinear',penalty='l1')  #LinearSVC()
    
    pred=numpy.zeros(len(y))
    pred_prob=numpy.zeros(len(y))


    for train,test in kf:
        clf.fit(mirt_data.iloc[train,:].values,y[train])
        if hasattr(clf,'predict_proba'):
            pred_prob.flat[test]=clf.predict_proba(mirt_data.iloc[test,:].values)
        pred.flat[test]=clf.predict(mirt_data.iloc[test,:].values)
    
    rocauc=roc_auc_score(y,pred_prob)
    print('%s:'%binary_vars[i])

    if numpy.var(pred)==0:
        print('WARNING: no variance in classifier output, degenerate model fit')
    if hasattr(clf,'predict_proba'):
        print('predictive accuracy (AUC: chance = 0.5) = %0.3f'%rocauc)
    else:
        print('Predictive accuracy')
    print(classification_report(y,pred,labels=[0,1]))
    print_confusion_matrix(y,pred)
    print("Features sorted by their absolute correlation with outcome (top %d):"%nfeatures)
    featcorr=numpy.array([numpy.corrcoef(mirt_data.iloc[:,x],y)[0,1] for x in range(mirt_data.shape[1])])
    idx=numpy.argsort(numpy.abs(featcorr))[::-1]
    for i in range(nfeatures):
        print('%f: %s'%(featcorr[idx[i]],mirt_vars[idx[i]]))


ArrestedChargedLifeCount:
predictive accuracy (AUC: chance = 0.5) = 0.515
             precision    recall  f1-score   support

          0       0.82      0.98      0.89       125
          1       0.00      0.00      0.00        26

avg / total       0.68      0.81      0.74       151

Confusion matrix
		Predicted
		0	1
Actual	0	122	3
	1	26	0
Features sorted by their absolute correlation with outcome (top 5):
-0.175285: 8
-0.136896: 5
0.125679: 7
0.092727: 4
0.044416: 3

DivorceCount:
predictive accuracy (AUC: chance = 0.5) = 0.558
             precision    recall  f1-score   support

          0       0.88      1.00      0.94       132
          1       1.00      0.05      0.10        19

avg / total       0.90      0.88      0.83       151

Confusion matrix
		Predicted
		0	1
Actual	0	132	0
	1	18	1
Features sorted by their absolute correlation with outcome (top 5):
0.219680: 9
-0.217268: 5
0.091259: 6
-0.081314: 4
0.066809: 1

GamblingProblem:
predictive accuracy (AUC: chance = 0.5